### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
5,5,Bryan Miranda,M,9th,Huang High School,94,94,0,District,2917,1910635
6,6,Sheena Carter,F,11th,Huang High School,82,80,0,District,2917,1910635
7,7,Nicole Baker,F,12th,Huang High School,96,69,0,District,2917,1910635
8,8,Michael Roth,M,10th,Huang High School,95,87,0,District,2917,1910635
9,9,Matthew Greene,M,10th,Huang High School,96,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
total_schools = len(school_data_complete["school_name"].value_counts())
total_students = school_data_complete["student_name"].count()
total_budget = school_data_complete["budget"].unique().sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_reading_score = school_data_complete["reading_score"].mean()

# Percentage for math
math_score_over_70 = school_data_complete.apply(lambda x: True if x["math_score"] >= 70 else False, axis=1)
math_score_over_70_count = len(math_score_over_70[math_score_over_70 == True].index)
percent_math_score_over_70 = (math_score_over_70_count / total_students)*100

# Percentage for reading
reading_score_over_70 = school_data_complete.apply(lambda y: True if y["reading_score"] >= 70 else False, axis=1)
reading_score_over_70_count = len(reading_score_over_70[reading_score_over_70 == True].index)
percent_reading_score_over_70 = (reading_score_over_70_count / total_students)*100

# # Percentage for overall passing
# overall_over_70 = school_data_complete.apply(lambda z: True if z["math_score"] >= 70 & z["reading_score"] >= 70 else False, axis=1)
# overall_over_70_count = len(overall_over_70[overall_over_70 == True].index)
# percent_overall = (overall_over_70_count / total_students)*100
# percent_overall

overall_over_70 = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & 
                                           (school_data_complete["math_score"] >= 70)].count()
percent_overall = (overall_over_70 / total_students)*100

district_summary = {"Total Schools": total_schools,
                    "Total Students": total_students, 
                    "Total Budget": total_budget, 
                    "Average Math Score": avg_math_score, 
                    "Average Reading Score": avg_reading_score, 
                    "% Passing Math": percent_math_score_over_70, 
                    "% Passing Reading": percent_reading_score_over_70, 
                    "% Overall Passing": percent_overall[1]}
district_summary_df = pd.DataFrame(district_summary, index = [0])
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:,.3f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:,.3f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.3f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.3f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.3f}%".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985,81.878,74.981%,85.805%,65.172%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [205]:
school_data_grouped = school_data_complete.groupby("school_name").agg({"type": "unique", "student_name": "count", 
                                                                       "math_score": "mean", "reading_score": "mean"
                                                                      })
                                        
x = school_data_grouped.reset_index()

total_budget = school_data_complete.groupby(["school_name", "budget"]).sum().reset_index()
totat_school_budget = list(total_budget["budget"])

student_counts = list(school_data_grouped["student_name"])
per_student_budget = [j/i for i, j in zip(student_counts, totat_school_budget)]

# % Passing Math
math_pass_per_school = school_data_complete.groupby(["school_name", 
                                                     school_data_complete["math_score"] >= 70])
y = list(math_pass_per_school.count()["Student ID"])
num_of_pass_math = y[1::2]
percent_math_per_school = [i/j*100 for i, j in zip(num_of_pass_math, student_counts)]

# % Passing Reading
reading_pass_per_school = school_data_complete.groupby(["school_name", 
                                                     school_data_complete["reading_score"] >= 70])
z = list(reading_pass_per_school.count()["Student ID"])
num_of_pass_reading = z[1::2]
percent_reading_per_school = [(a/b)*100 for a, b in zip(num_of_pass_reading, student_counts)]

# % Overall Passing
overall_pass_per_school = school_data_complete.groupby(["school_name", 
                                                     school_data_complete["reading_score"] & school_data_complete["math_score"] >= 70])
k = list(overall_pass_per_school.count()["Student ID"])
num_of_overall_pass = k[1::2]
percent_overall_pass_per_school = [(c/d)*100 for c, d in zip(num_of_overall_pass, student_counts)]

school_data_grouped_df = pd.DataFrame(school_data_grouped)
school_data_grouped_df["Per Student Budget"] = per_student_budget
school_data_grouped_df["% Passing Math"] = percent_math_per_school
school_data_grouped_df["% Passing Reading"] = percent_reading_per_school
school_data_grouped_df["% Overall Passing"] = percent_overall_pass_per_school
school_data_grouped_df["Per Student Budget"] = school_data_grouped_df["Per Student Budget"].map("${:.2f}".format)
school_data_grouped_df.rename(columns={"type": "School Type", "student_name": "Total Students", 
                                       "math_score": "Average Math Score", "reading_score": "Average Reading Score"})



,School Type,Total Students,Average Math Score,Average Reading Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,
Bailey High School,[District],4976,77.048432,81.033963,$628.00,66.680064,81.933280,30.345659
Cabrera High School,[Charter],1858,83.061895,83.975780,$582.00,94.133477,97.039828,50.861141
Figueroa High School,[District],2949,76.711767,81.158020,$639.00,65.988471,80.739234,30.111902
Ford High School,[District],2739,77.102592,80.746258,$644.00,68.309602,79.299014,30.485579
Griffin High School,[Charter],1468,83.351499,83.816757,$625.00,93.392371,97.138965,50.136240
Hernandez High School,[District],4635,77.289752,80.934412,$652.00,66.752967,80.862999,31.024811
Holden High School,[Charter],427,83.803279,83.814988,$581.00,92.505855,96.252927,49.648712
Huang High School,[District],2917,76.629414,81.182722,$655.00,65.683922,81.316421,29.516627
Johnson High School,[District],4761,77.072464,80.966394,$650.00,66.057551,81.222432,30.182735


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type